In [1]:
# https://blog.csdn.net/marsjhao/article/details/68490105 https://blog.csdn.net/u014281392/article/details/77103747
import numpy as np
import scipy.io as sio
np.random.seed(355)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras import layers
from keras.layers import Dropout, Bidirectional,Activation,Flatten,GRU,LSTM
from keras.layers import Dense, Input, TimeDistributed,RepeatVector
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras import Model
from keras.optimizers import Adam
import math
import time
import config_seq_hep as cf
import six.moves.cPickle as pickle

C:\Users\fengx\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
C:\Users\fengx\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\fengx\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\fengx\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type

observation time 3599
the number of time interval: 6
time interval: 600


In [2]:
# 全局变量
batch_size = 32
epochs = 20
CELL_SIZE=32
OUTPUT_SIZE=16
output_length = len(cf.pre_times)
# input image dimensions
TIME_STEPS = cf.n_time_interval
INPUT_SIZE = len(cf.degree_interval_list)+1
input_shape = (TIME_STEPS, INPUT_SIZE) 

In [3]:
id_train, x_train, L, y_train, sz_train, time_train, vocabulary_size = pickle.load(open(cf.train_pkl, 'rb'))
id_test, x_test, L_test, y_test, sz_test, time_test, _ = pickle.load(open(cf.test_pkl, 'rb'))
id_val, x_val, L_val, y_val, sz_val, time_val, _ = pickle.load(open(cf.val_pkl, 'rb'))
for i in range(len(x_train)):
    x_train[i] = np.array(x_train[i])
    y_train[i] = np.array(y_train[i])
for i in range(len(x_val)):
    x_val[i] = np.array(x_val[i])
    y_val[i] = np.array(y_val[i])
for i in range(len(x_test)):
    x_test[i] = np.array(x_test[i])
    y_test[i] = np.array(y_test[i])
x_val = np.array(x_val)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_val = np.array(y_val)
y_train = np.array(y_train)
y_test = np.array(y_test)

y_val=y_val.reshape(y_val.shape[0],output_length,1)
y_train=y_train.reshape(y_train.shape[0],output_length,1)
y_test=y_test.reshape(y_test.shape[0],output_length,1)

print(x_train.shape)
print(y_train.shape)
# x_train = x_train.reshape(x_train.shape[0], TIME_STEPS, INPUT_SIZE, 1)
# x_test = x_test.reshape(x_test.shape[0], TIME_STEPS, INPUT_SIZE, 1)
# x_val = x_val.reshape(x_val.shape[0], TIME_STEPS, INPUT_SIZE, 1)

(29529, 6, 10)
(29529, 8, 1)


In [6]:
def mse_compute(y_pred,y):
    y_pred = y_pred[:,-1,0]
    y = y[:,-1,0]
    mse = np.mean(np.power(y-y_pred,2))
    return mse

In [20]:
## LSTM Seq2Seq
def seq2seq_model(input_length,output_sequence_length):
    model = Sequential()
    model.add(Bidirectional(GRU(32, return_sequences = False)))
    model.add(Dense(16, activation="tanh"))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(32, return_sequences = True)))
    model.add(Dense(16, activation="tanh"))
    model.add(TimeDistributed(Dense(1, activation = 'relu')))
    model.compile(loss='mean_squared_error',optimizer=Adam(1e-3),metrics=['mse'])
    model.build((None,input_length,INPUT_SIZE))
    model.summary()
    return model
model = seq2seq_model(x_train.shape[1],y_train.shape[1])

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc', patience = 3,verbose = 1, factor=0.5, min_lr = 0.00001)
#训练模型
f = open('result_epoch_seq2seq.csv','w')
for epoch in range(epochs):
    model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=1, validation_data=(x_val, y_val), 
              shuffle=True, verbose=1,callbacks=[learning_rate_reduction])
    #score = model.evaluate(x_test, y_test, verbose=0)
    y_test_predic = model.predict(x_test)
    test_score = mse_compute(y_test_predic,y_test)
    y_val_predic = model.predict(x_val)
    val_score = mse_compute(y_val_predic,y_val)
    print('Test score:', test_score,'\t','Val_score',val_score)
    f.write(str(epoch)+',Test score,'+ str(test_score[0])+',Val_score,'+str(val_score[0])+'\n')
f.close()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_9 (Bidirection (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 16)                1040      
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 8, 16)             0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 8, 64)             9408      
_________________________________________________________________
dense_12 (Dense)             (None, 8, 16)             1040      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 8, 1)              17        
Total params: 19,761
Trainable params: 19,761
Non-trainable params: 0
__________________________________________________

C:\Users\fengx\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 10s 330us/step - loss: 2.3459 - mse: 2.3459 - val_loss: 1.9710 - val_mse: 1.9710


C:\Users\fengx\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_mse,loss,mse,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Test score: 2.0941842257991605 	 Val_score 2.2491521145626883
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 9s 290us/step - loss: 1.8351 - mse: 1.8351 - val_loss: 1.9166 - val_mse: 1.9166
Test score: 2.030105534017773 	 Val_score 2.1877624989814857
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 8s 286us/step - loss: 1.8065 - mse: 1.8065 - val_loss: 1.9370 - val_mse: 1.9370
Test score: 2.047245500293961 	 Val_score 2.2167919367412874
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 11s 376us/step - loss: 1.7886 - mse: 1.7886 - val_loss: 1.9031 - val_mse: 1.9031
Test score: 2.039233059227877 	 Val_score 2.166288080812645
Train on 29529 samples, validate on 6327 samples
Epoch 1/1
29529/29529 [==============================] - 12s 419us/step - loss: 1.7707 - mse: 1.7707 - val_loss: 1.9735 - val_mse: 1.9735
Test score

In [17]:
score = model.evaluate(x_test, y_test, verbose=0)
y_test_pre = model.predict(x_test)
print('Test score:', score[0])
f = open('result_test_seq2seq.csv','w')
for i in range (len(y_test)):
    f.write(str(y_test[i,-1,0])+','+str(y_test_pre[i,-1,0])+'\n')
f.close()

Test score: 1.6536562992570447


In [13]:
output_length

8

In [16]:
y_train_predic= model.predict(x_train)
train_score= mse_compute(y_train_predic,y_train)
print (y_train_predic[358])
print (y_train[358])
print (train_score)

[[4.7871284]
 [5.416479 ]
 [5.7073374]
 [5.8398314]
 [5.9066963]
 [5.9464417]
 [5.9948688]
 [6.089055 ]]
[[5.67242534]
 [5.83289001]
 [5.83289001]
 [5.857981  ]
 [5.857981  ]
 [5.9068906 ]
 [5.93073734]
 [5.93073734]]
1.8775853188032454
